In [1]:
import sys
from pyscf.lib import logger
from pyscf import gto, scf
import numpy as np
from functools import reduce
_fdot = np.dot
fdot = lambda *args: reduce(_fdot, args)

In [2]:
log = logger.Logger(sys.stdout, 6)

a = 1 
nH = 10
atoms1 = ""
for i in range(nH):
    atoms1 += f"H {i*a:.5f} 0.00000 0.00000 \n"

b = 1.5
atoms2 = ""
for i in range(nH):
    if i < nH-1:
        atoms2 += f"H {i*a:.5f} 0.00000 0.00000 \n"
    else:
        atoms2 += f"H {(i-1)*a+b:.5f} 0.00000 0.00000 \n"

basis = "sto6g"
mol1 = gto.M(atom=atoms1,basis=basis,verbose=4)
mol2 = gto.M(atom=atoms2,basis=basis,verbose=4)

mf1 = scf.RHF(mol1).density_fit()
mf1.kernel()

mf2 = scf.RHF(mol2).density_fit()
mf2.kernel()

e_mf1 = mf1.e_tot
e_mf2 = mf2.e_tot
print(f"mf energy difference: {e_mf1-e_mf2:.8f}")


System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#1882-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Sun Aug 17 20:51:55 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 10
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.



******** <class 'pyscf.df.df_jk.DFRHF'> ********
method = DFRHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = False
chkfile to save SCF result = /tmp/tmpnml3aoyo
max_memory 4000 MB (current use 115 MB)
Set gradient conv threshold to 3.16228e-05
Initial guess from minao.
******** <class 'pyscf.df.df.DF'> ********
auxbasis = None
max_memory = 4000
ETB for H: l = 0, exps = 0.200224856 * 2^n , n = 0..8

WARN: Even tempered Gaussians are generated as DF auxbasis for  H

init E= -4.16247989776679
  HOMO = -0.143333294554415  LUMO = 0.0777539909476834
cycle= 1 E= -5.23268311405475  delta_E= -1.07  |g|= 0.127  |ddm|= 2.26
  HOMO = -0.255531209164804  LUMO = 0.134258263865682
cycle= 2 E= -5.2469967389458  delta_E= -0.0143  |g|= 0.057  |ddm|= 0.341
  HOMO = -0.266117832608071  LUMO = 0.161373699495237
cycle= 3 E

In [ ]:
options = {
    "dt": 0.01,
    "n_exp_terms": 6,
    "n_walkers": 20,
    "n_runs": 50,
    "n_prop_steps": 10,
    "rlx_steps": 3,
    "prop_steps": 10,
    "seed": 23,
    "walker_type": "rhf",
    "trial": "rhf",
    "use_gpu": False,
    "free_proj": False,
}

from ad_afqmc.corr_sample import lnocs
lnocs.run_cs_frags(mf1,mf2,options=options,lno_thresh=1e-3,nproc=4,run_frg_list=None)

lo_type = pm
lo_type = pm

########### running fragment 1 ##########
# Preparing CS-LNO-AFQMC calculation
# Generating Cholesky Integrals
# correlated sampling requires Density Fitting!
# Decomposing ERI with DF
# Finished calculating Cholesky integrals

# Size of the correlation space:
# Number of electrons: (1, 1)
# Number of basis functions: 3
# Number of Cholesky vectors: 90
# Active orbitals: [4, 5, 6]
# frozen orbitals: [0 1 2 3 7 8 9]
# Preparing CS-LNO-AFQMC calculation
# Generating Cholesky Integrals
# correlated sampling requires Density Fitting!
# Decomposing ERI with DF
# Finished calculating Cholesky integrals

# Size of the correlation space:
# Number of electrons: (1, 1)
# Number of basis functions: 3
# Number of Cholesky vectors: 90
# Active orbitals: [4, 5, 6]
# frozen orbitals: [0 1 2 3 7 8 9]
# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x8

In [12]:
with open('results.out', 'w') as out_file:
    print('# frag' \
            '  orb_energy1  err  orb_energy2  err' \
            '  d_orb_energy  cs_err  time',file=out_file)
    for ifrag in range(5):
        with open(f"cs_frg{ifrag+1}.out","r") as read_file:
            for line in read_file:
                if "orbital energy 1" in line:
                    orb_energy1 = line.split()[4]
                    orb_err1 = line.split()[6]
                    print(orb_energy1)

-0.017465
-0.018512
-0.020912
-0.022894
-0.016815


In [ ]:
with open('results.out', 'w') as out_file:
    print('# frag' \
            '  orb_energy1  err  orb_energy2  err' \
            '  d_orb_energy  cs_err  orb_mp2_1 orb_mp2_2' \
            '  runtime',file=out_file)
    for ifrag in range(5):
        with open(f"cs_frg{ifrag+1}.out","r") as read_file:
            for line in read_file:
                if "orbital energy 1" in line:
                    orb_energy1 = line.split()[4]
                    orb_err1 = line.split()[6]
                if "orbital energy 2" in line:
                    orb_energy2 = line.split()[4]
                    orb_err2 = line.split()[6]
                if "correlated d_energy" in line:
                    d_orb_en = line.split()[3]
                    cs_err = line.split()[5]
                if "lno-mp2 orb_corr1" in line:
                    orb_mp21 = line.split()[3]
                if "lno-mp2 orb_corr2" in line:
                    orb_mp22 = line.split()[3]
                if "total run time" in line:
                    runtime = line.split()[4]
            print(f'{ifrag+1:3d}  '
                  f'{orb_energy1}  {orb_err1}  '
                  f'{orb_energy2}  {orb_err2}  '
                  f'{d_orb_en}  {cs_err}  '
                  f'{orb_mp21}  {orb_mp22}  '
                  f'{runtime}', file=out_file)


In [ ]:
data = []
with open('results.out', 'r') as read_file:
    for line in read_file:
        if not line.startswith("#"):
            data = np.hstack((data,line.split()))
            data[data == 'None'] = '0.000000' 

data = np.array(data.reshape(5,10))
orb_en1 = np.array(data[:,1],dtype='float32')
orb_err1 = np.array(data[:,2],dtype='float32')
orb_en2 = np.array(data[:,3],dtype='float32')
orb_err2 = np.array(data[:,4],dtype='float32')
d_orb_en = np.array(data[:,5],dtype='float32')
cs_err = np.array(data[:,6],dtype='float32')
orb_mp21 = np.array(data[:,7],dtype='float32')
orb_mp22 = np.array(data[:,8],dtype='float32')
orb_time = np.array(data[:,9],dtype='float32')

tot_corr1 = sum(orb_en1)
tot_err1 = np.sqrt(sum(orb_err1**2))
tot_corr2 = sum(orb_en2)
tot_err2 = np.sqrt(sum(orb_err2**2))
d_tot_corr = sum(d_orb_en)
cs_tot_err = np.sqrt(sum(cs_err**2))
tot_mp21 = sum(orb_mp21)
tot_mp22 = sum(orb_mp22)
tot_time = sum(orb_time)

mp2_cr1 = 

In [ ]:
benzaldehyde='''
C	0	0	0
C	-1.044059	-0.954468	0
C	-0.735914	-2.325901	0
C	0.613848	-2.747014	0
C	1.655764	-1.796732	0
C	1.347187	-0.42102	0
C	-0.327447	1.46041	0
O	-1.463198	1.909723	0
H	0.567159	2.139658	0
H	-2.081762	-0.59926	0
H	-1.541618	-3.071175	0
H	0.852246	-3.818652	0
H	2.702313	-2.126863	0
H	2.150974	0.328744	0
'''